In [1]:
from datetime import datetime
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px

## Información financiera de empresas S&P 500 

In [2]:
# Obtener informacion de las empresas S&P 500
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [3]:
# Definir los tickers que se analizaran
symbols_sp500 = list(sp500['Symbol'])
symbols_sp500[symbols_sp500.index('BRK.B')] = 'BRK-B'      # Actualización por error en la descarga: BRK.B: No data found, symbol may be delisted
symbols_sp500[symbols_sp500.index('BF.B')] = 'BF-B'       # Actualización por error en la descarga: BF.B: No data found for this date range, symbol may be delisted

# Obtener datos con el siguiente intervalo
start_date = '2000-01-01'
end_date = '2021-12-31'

# Descargar los datos financieros 
data = yf.download(symbols_sp500, start_date, end_date)

[*********************100%***********************]  503 of 503 completed

1 Failed download:
- CEG: Data doesn't exist for startDate = 946706400, endDate = 1640930400


In [4]:
# Copia de datos financieros ordenada por fecha 
data_copy = data.copy()
data_copy.sort_index(inplace=True)
data_copy.head()

Adj Close                                             \
                             A AAL AAP      AAPL ABBV       ABC      ABMD   
Date                                                                        
2000-01-03 00:00:00  44.143158 NaN NaN  0.854541  NaN  3.016693  18.25000   
2000-01-04 00:00:00  40.771107 NaN NaN  0.782495  NaN  2.810734  17.81250   
2000-01-05 00:00:00  38.242062 NaN NaN  0.793945  NaN  3.028807  18.00000   
2000-01-06 00:00:00  36.785961 NaN NaN  0.725238  NaN  3.258995  18.03125   
2000-01-07 00:00:00  39.851456 NaN NaN  0.759592  NaN  3.683029  17.93750   

                                              ... Volume             \
                          ABT ACN       ADBE  ...   WYNN        XEL   
Date                                          ...                     
2000-01-03 00:00:00  9.347845 NaN  16.274675  ...    NaN  2738600.0   
2000-01-04 00:00:00  9.080765 NaN  14.909399  ...    NaN   425200.0   
2000-01-05 00:00:00  9.064073 NaN  15.204174  ...    NaN   500200.0   
2000-01-06 00:00:00  9.381229 NaN  15.328291  ...    NaN   344100.0   
2000-01-07 00:00:00  9.481389 NaN  16.072979  ...    NaN   469500.0   

                                                                          \
                            XOM       XRAY XYL        YUM ZBH       ZBRA   
Date                                                                       
2000-01-03 00:00:00  13458200.0   582300.0 NaN  3033493.0 NaN  1055700.0   
2000-01-04 00:00:00  14510800.0   317700.0 NaN  3315031.0 NaN   522450.0   
2000-01-05 00:00:00  17485000.0  1188000.0 NaN  4642602.0 NaN   612225.0   
2000-01-06 00:00:00  19461600.0   534300.0 NaN  3947658.0 NaN   263925.0   
2000-01-07 00:00:00  16603800.0  1401000.0 NaN  6063647.0 NaN   333900.0   

                                    
                          ZION ZTS  
Date                                
2000-01-03 00:00:00  1199600.0 NaN  
2000-01-04 00:00:00   816100.0 NaN  
2000-01-05 00:00:00  1124700.0 NaN  
2000-01-06 00:00:00  1112100.0 NaN  
2000-01-07 00:00:00   782000.0 NaN  

[5 rows x 3018 columns]

In [5]:
# Datos financieros por cada ticker
list(data_copy.columns.levels[0])

['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

## Indicadores financieros

In [6]:
gap_return = np.log(data_copy['Open']/data_copy['Close'].shift(1)).fillna(0)
intra_return = np.log(data_copy['Close']/data_copy['Open']).fillna(0)
variation = data_copy['Adj Close'].pct_change()
volatility = (data_copy['Adj Close'].pct_change()).rolling(250).std()*100*(250)**0.5

### Mejor día para invertir teniendo en cuenta el retorno de los movimiento gap

In [7]:
gap_return['Week Day'] = gap_return.index.map(lambda x: x.weekday())
gap_return_day = pd.DataFrame(gap_return.groupby('Week Day').sum().T.mean(), columns=['Gap Return'])
gap_return_day = gap_return_day.rename(index={0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes'})
gap_return_day

,Gap Return
Week Day,
Lunes,0.072118
Martes,0.583219
Miércoles,0.116780
Jueves,0.056534
Viernes,0.219348


In [8]:
fig = px.bar(gap_return_day, x=gap_return_day.index, y='Gap Return', title='Retorno de movimietos gap')
fig.show()

### Mejor día para invertir teniendo en cuenta el retorno de los movimientos intradiarios

In [9]:
intra_return['Week Day'] = intra_return.index.map(lambda x: x.weekday())
intra_return_day = pd.DataFrame(intra_return.groupby('Week Day').sum().T.mean(), columns=['Intra Return'])
intra_return_day = intra_return_day.rename(index={0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes'})
intra_return_day

,Intra Return
Week Day,
Lunes,-0.284916
Martes,0.189122
Miércoles,0.241616
Jueves,0.516847
Viernes,0.246834


In [10]:
fig = px.bar(intra_return_day, x=intra_return_day.index, y='Intra Return', title='Retorno de movimietos intradiarios')
fig.show()

### Mejores industrias que pertenecen al SP500 en las cuales se puede invertir

In [11]:
ticker_growth = (((data_copy['Close'] - data_copy['Open']) / data_copy['Open']) * 100).sum()
ticker_growth = pd.DataFrame(ticker_growth, columns=['Crecimiento'])
ticker_growth = ticker_growth.merge(sp500, left_on=ticker_growth.index, right_on='Symbol')
ticker_growth.drop(ticker_growth.columns.difference(['Crecimiento','Symbol', 'GICS Sector']), 1, inplace=True)
sector_growth = ticker_growth.groupby('GICS Sector').mean()
sector_growth.sort_values('Crecimiento', ascending=False, inplace=True)
sector_growth

/var/folders/5w/5qvg4ht11gqbxjjz9hc2zbs80000gn/T/ipykernel_22851/655384077.py:4: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



,Crecimiento
GICS Sector,
Health Care,269.043737
Real Estate,250.821516
Consumer Staples,243.956007
Information Technology,235.067028
Industrials,208.514808
Financials,182.737073
Utilities,170.773221
Consumer Discretionary,169.633466
Materials,108.468926


In [12]:
fig = px.bar(sector_growth, x=sector_growth.index, y='Crecimiento', title='Mejores industrias que pertenecen al SP500')
fig.show()

### Momentos de alta volatilidad que afectaron al SP500

In [13]:
volatility_avg = volatility.T.mean()
fig = px.line(x=volatility_avg.index, y=volatility_avg, labels={'x': 'Year', 'y':'Volatility'}, title="Momentos de volatilidad")
fig.show()

### 9 mejores empresas para invertir

In [14]:
ticker_growth = (((data_copy['Close'] - data_copy['Open']) / data_copy['Open']) * 100).sum()
ticker_growth = pd.DataFrame(ticker_growth, columns=['Crecimiento'])
ticker_growth = ticker_growth.merge(sp500, left_on=ticker_growth.index, right_on='Symbol')
ticker_growth.drop(ticker_growth.columns.difference(['Crecimiento','Symbol', 'Security']), 1, inplace=True)
ticker_growth = ticker_growth.groupby('Security').mean()
ticker_growth = ticker_growth.sort_values(by='Crecimiento', ascending=False)
top_nine = ticker_growth[:9]
top_nine

/var/folders/5w/5qvg4ht11gqbxjjz9hc2zbs80000gn/T/ipykernel_22851/662837639.py:4: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



,Crecimiento
Security,
"NVR, Inc.",868.977961
Idexx Laboratories,790.209597
Ansys,774.107000
Nordson Corporation,760.147243
Accenture,755.444581
Gartner,714.626904
Autodesk,685.686491
Cadence Design Systems,671.674672
Monster Beverage,665.077244


In [15]:
fig = px.bar(top_nine, x=top_nine.index, y='Crecimiento', labels={'Security':'Ticker'}, title='Mejores empresas para invertir')
fig.show()